# Feature Extractor Data Augmentation

Lo scopo di questo Notebook Python è di eseguire una breve analisi sui dati usati per addestrare il feature extractor, che servirà da base per la codifica delle immagini nel modello finale del KvasirVQA, e in particolare alle operazioni di data augmentation effettuate per ribilanciare le classi meno rappresentate.

In [1]:
from loader import prepare_data
from loader import format_float as f
import matplotlib.pyplot as plt
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\emanu\\Desktop\\Tesi\\kvasir-vqa\\notebooks\\logs\\2025-01-05.log'

In [ ]:
dataset = prepare_data(os.getenv('FEATURE_EXTRACTOR_CSV'))

N = dataset.shape[0]
labels = list(set(dataset["label"]))

dataset.head()

In [ ]:
dataset.describe()

I dati usati (combinazione del dataset Hyper Kvasir e Kvasir Instrument) arrivano a circa immagini etichettate, con la presenza di bounding box per quelle che interessano la categoria dei polipi e degli strumenti.

Il dataset non è generalmente bilanciato per un problema intrinseco dei dati, ovvero il fatto che alcuni ritrovamenti e patologie appaiono più frequentemente di altri.

In [ ]:
labels_count = {}

for label in labels:
    labels_count[label] = len(dataset[dataset['label'] == label])

labels_count = dict(sorted(labels_count.items(), key=lambda item: item[1], reverse=True))

for l in labels_count:
    print(f"Label: {l}, n: {labels_count[l]}/{N} - {f(labels_count[l]/N * 100)}%")

A causa di una presenza di sbilanciamento nella distribuzione dei dati tra le classi, si è scelto di fare data augmentation sulle classi con presenza più bassa, in particolare su quelle classi con meno di 300 campioni presenti.

In [ ]:
labels = labels_count.keys()
values = labels_count.values()

x = np.arange(len(labels))

fig, ax = plt.subplots()
ax.barh(x, values, align='center')

ax.set_yticks(x, labels=labels)
ax.invert_yaxis()
ax.set_xlabel("Samples")

plt.show()

Il codice per la data augmentation è presente nel file dataset.py, sotto il metodo __prepare_feautre_extractor_data__.

Facciamo una veloce analisi dei dati come fatta subito prima ma per il dataset comprensivo di immagini aumentate.

In [ ]:
aug_dataset = prepare_data(os.getenv('FEATURE_EXTRACTOR_CSV_AUG'), aug=True)

aug_N = aug_dataset.shape[0]
aug_labels = list(set(aug_dataset["label"]))

aug_dataset.head()

In [ ]:
aug_dataset.describe()

In [ ]:
aug_labels_count = {}

for label in aug_labels:
    aug_labels_count[label] = len(aug_dataset[aug_dataset['label'] == label])

aug_labels_count = dict(sorted(aug_labels_count.items(), key=lambda item: item[1], reverse=True))

for l in aug_labels_count:
    print(f"Label: {l}, n: {aug_labels_count[l]}/{aug_N} - {f(aug_labels_count[l]/aug_N * 100)}%")

In [ ]:
aug_values = aug_labels_count.values()

x = np.arange(len(labels))

fig, ax = plt.subplots()
ax.barh(x, aug_values, align='center')

ax.set_yticks(x, labels=labels)
ax.invert_yaxis()
ax.set_xlabel("Samples")

plt.show()

Dopo aver fatto data augmentation applicando piccole rotazioni casuali alle immagini e ribaltamenti orizzontali, ci ritroviamo di fronte a un insieme più bilanciato ed omogeneo per l'addestramento.